# Error Analysis

*Author: Alex Lewandowski; Alaska Satellite Facility*

This notebook assumes that you have a completed MintPy time series analysis.

---

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">
      <img src="github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">opensarlab_MintPy_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, ASF data access, or performing SBAS time series analyses with MintPy?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---

## 0. Import Required Software

In [ ]:
from pathlib import Path
import sys

from ipyfilechooser import FileChooser
import ipywidgets as widgets
from ipywidgets import Layout
import opensarlab_lib as osl
from shapely.geometry import Point, box

from mintpy.cli import view
from mintpy.utils import plot as pp

current = Path("..").resolve()
sys.path.append(str(current))
import util.gps as gps
import util.util as util

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [ ]:
file_chooser_path = util.get_recent_mintpy_config_path()
if file_chooser_path:
    fc = FileChooser(path=file_chooser_path.parent, filename=file_chooser_path.name, select_default=True)
else:
    file_chooser_path = Path.home()
    fc = FileChooser(file_chooser_path, select_default=False)
    
print("Select your custom MintPy config file (MintPy/my_project_name.txt):")
display(fc)

In [ ]:
if Path(fc.selected) != file_chooser_path:
    util.write_recent_mintpy_config_path(Path(fc.selected))

config_path = Path(fc.selected)
mint_path = config_path.parent
plot_path = mint_path / "plots"

## 1. Spatial Coherence

Spatial Coherence measures phase consistency across neighborhoods of pixels in each interferogram in the stack. A high level of spatial coherence shows that there was little phase change across adjacent pixels. 

In [ ]:
%matplotlib widget
scp_args = f"{mint_path}/avgSpatialCoh.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/avg_spatial_coh.png"
view.main(scp_args.split())

## 2. Temporal Coherence

Temporal coherence is a measure of phase consistency for a pixel across the time series. A high level of temporal coherence indicates that phase changes in a pixel were steady, providing a measure of reliability. 

- [0, 1] range
- 0 = unreliable, 1 = reliable
- recommended threshold for dense time series: 0.7 (Yunjun et al, 2019)

In [ ]:
%matplotlib widget
scp_args = f"{mint_path}/temporalCoherence.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/temporal_coh.png"
view.main(scp_args.split())

---
## 3. Velocity error analysis

The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

- Velocity error increases with distance from the reference point
- Can indicate atmospheric delay

In [ ]:
%matplotlib widget

cmap = 'Reds'
scp_args = f'{mint_path}/velocity.h5 velocityStd --dpi 600 --figsize 15 15 --colormap {cmap} --outfile {plot_path}/velocity_err.png'
view.main(scp_args.split())

---
## 4. Compare InSAR time-series with GPS time-series in LOS direction

- https://geodesy.unr.edu/NGLStationPages/gpsnetmap/GPSNetMap.html
- https://geodesy.unr.edu/NGLStationPages/DataHoldings.txt

**Access University of Nevada, Reno GPS station metadata and select a list of available stations** 

Ignore stations:
- outside the AOI
- outside the time range of the time series
- on no-data pixels

Two GPS stations are required for LOS conversion

<br>
<div class="alert alert-warning">
<font face="Calibri" size="5"><b><font color='rgba(200,0,0,0.2)'> <u>Supports Data in a Single UTM Only</u></font></b></font>

<font face="Calibri" size="3">MintPy's `view.main` `--show-gps` option currently only supports datasets that fall in a single UTM zone. Many Sentinel-1 scenes fall across multiple UTM zones and are projected to the predominantly covered zone. There is currently an assumption in the code that all lat/lon coordinates in the scene will be projected to the same CRS when using `utm.from_latlon`, which is not always the case.
</font>
</div>


In [ ]:
# Download DataHolding.txt and write to GPS_stations.csv
# in MintPy directory
gps.create_unr_gps_csv(mint_path)

# Finds stations in the AOI and time span of the time series.
# Ignore stations with no time series velocity data
gps_stations = gps.get_gps_stations(mint_path)
gps_stations

**Create interactive plot showing GPS station information**

Hover over GPS stations to view their details

In [ ]:
if gps_stations:
    gps_dict = gps.get_gps_dict(mint_path, gps_stations)
    velocity_png_pth = plot_path / "velocity.png"
    print("Hover over GPS stations to view their details")
    gps.gps_station_info_plot(mint_path, velocity_png_pth, gps_stations, gps_dict)

**Select a reference GPS station** 

In [ ]:
gps_station = widgets.RadioButtons(
    options=gps_stations,
    description='',
    disabled=False,
    layout=Layout(min_width='800px'))

print("Select a GPS station")
display(gps_station)

**Run the LOS comparison with the selected reference GPS station**

You can select different reference stations and rerun the following code cell multiple times to see results referenced to different areas of your time series. 

In [ ]:
%matplotlib widget

vmin, vmax = util.get_mintpy_vmin_vmax(mint_path/'velocity.h5', mask_path=mint_path/'maskTempCoh.h5',  bottom_percentile=0.05)

scp_args = f"{mint_path}/velocity.h5 velocity -v {vmin} {vmax} --show-gps --ref-gps {gps_station.value} --gps-comp enu2los --gps-label --figsize 9 9"
with osl.work_dir(mint_path):
    try:
        view.main(scp_args.split())
    except IndexError:
        raise Exception(
            ("""
            IndexError: Likely caused by data in more than one UTM.
            MintPy's view.main --show-gps option currently only supports datasets that fall in a single UTM zone. 
            Many Sentinel-1 scenes fall across multiple UTM zones and are projected to the predominantly covered zone. 
            There is currently an assumption in the code that all lat/lon coordinates in the scene will be projected to the same CRS when using utm.from_latlon, which is not always the case.
            """)
        )

In [ ]:
gps_enu2los_path = mint_path / 'gps_enu2los.csv'
vel_path = mint_path / 'velocity.h5'
msk_path = mint_path / 'maskTempCoh.h5'
with osl.work_dir(plot_path):
    pp.plot_insar_vs_gps_scatter(
        vel_path,
        ref_gps_site=gps_station.value,
        csv_file=gps_enu2los_path,
        msk_file=msk_path,
        # vlim=[-2.5, 2], # option to prune sites that fall outside of a value range (cm)
    )